In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

def proc_meteo_ish(df, meteorology_file, fechas):
    
    """
    Procesa desde los archivos que armó Ramiro a partir de 
    archivos .ISH. Debe existir una lista de fechas relevantes,
    puede estar en df.
    """
    
    df = pd.read_csv(meteorology_file, delimiter=';')
    df = df[(df['date[yyyymmddHHMM]'] >= 201904031200)]
    #fechas = pd.read_csv('fechas.csv')
    #fechas['df'] = pd.to_datetime(fechas['df'], format='%d/%m/%Y')
    # Reemplazo los valores que son nan
    df['wdir'] = df['wdir'].where(df['wdir'] < 999)
    df['wspd[m/s]'] = df['wspd[m/s]'].where(df['wspd[m/s]'] < 999)
    df['clht[km]'] = df['clht[km]'].where(df['clht[km]'] < 99)
    df['dptp[C]'] = df['dptp[C]'].where(df['dptp[C]'] < 999)
    df['slvp[hPa]'] = df['slvp[hPa]'].where(df['slvp[hPa]'] < 9999)
    df['press[hPa]'] = df['press[hPa]'].where(df['press[hPa]'] < 9999)
    df['prcp[mm]'] = df['prcp[mm]'].where(df['prcp[mm]'] < 999)
    df['sky[octas]'] = df['sky[octas]'].where(df['sky[octas]'] > 99)
    df['skyOpaque[octas]'] = df['skyOpaque[octas]'].where(df['skyOpaque[octas]'] < 99)
    df['tmpd[C]'] = df['tmpd[C]'].where(df['tmpd[C]'] < 99)
    
    # Calculo RH como rh=100*(EXP((17.625*d)/(243.04+d))/EXP((17.625*t)/(243.04+t)));
    # donde d=dewpoint (dptp) y t=dry temp (tmpd), aprox de August-Roche-Magnus
    d = df['dptp[C]']
    t = df['tmpd[C]']
    df['RH[%]'] = 100 * (np.exp((17.625 * d)/(243.04 + d)) / np.exp((17.625 * t)/(243.04 + t)))
    # 
    
    #Promedio fechas que están dobles
    df = df.groupby('date[yyyymmddHHMM]').mean().reset_index()
    
    
    
    # Parsing de fecha
    datetime = pd.to_datetime(df['date[yyyymmddHHMM]'], format='%Y%m%d%H%M')
    df['date'] = datetime
    # Reordeno columnas-
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1] # Mando la última columna (date) al principio
    df = df[cols]
    df = df.set_index('date')
    df = df.groupby(level=0).sum()
    df24hs_mean = df.resample('24H', offset='12h').mean() # Resampleo para tener promedios diarios
    df24hs_sum = df.resample('24H', offset='12h').sum() # Resampleo para tener promedios diarios
    df_comb = pd.concat([df24hs_mean[['wdir', 'wspd[m/s]', 'clht[km]', 'hzvs[km]', 'tmpd[C]',
                                              'slvp[hPa]', 'press[hPa]', 'sky[octas]', 'skyOpaque[octas]', 'RH[%]']],
                             df24hs_sum[['prcp[mm]', 'prcpPeriod[hours]']]], axis=1)
    
    df_comb.index = df_comb.index.normalize()
    df_comb = (df_comb.reindex(index = fechas))
    return df_comb


def plot_correl(df, meteo, medida, dir_salida):
    """
    Plotea las correlaciones necesarias y las guarda en dir_salida.
    meteo y medida son listas. df es la dataframe con los datos.
    """
    for i in meteo:
        for j in medida:
            fig, ax = plt.subplots()
            ax.plot(df[i], df[j], 'o', label=j)
            ax.set_xlabel(i)
            ax.set_ylabel(j)
            ax.legend()
            salida = j
            if '/' in salida:
                salida = salida.replace('/', '')
            if '.' in salida:
                salida = salida.replace('.', '')
            if ',' in salida:
                salida = salida.replace(',', '')
            if ' ' in salida:
                salida = salida.replace(' ', '_')
            fig.savefig(f'{dir_salida}/{salida}_vs_{i}.png')
            plt.close(fig)
    return 0

In [2]:
mexico = pd.read_excel('Datos_Pablo_paraprocesar.xlsx', sheet_name='Mexico', nrows=120, usecols="A:AE")
print(mexico.keys())
mexico = mexico.set_index('FECHA MUESTREO')
display(mexico)

Index([' Id. Filtro', 'FECHA MUESTREO', 'Volumen act (m3)', 'PM2.5 (ug/m3)',
       'Periodo Climático', 'Influencia regional', 'OC (µg/m3)', 'EC (µg/m3)',
       'TC (µg/m3)', 'OC/EC', 'Percent', 'OCprim20%', 'SOC', 'SOC/OC',
       'OCprim10%', 'SOC.1', 'SOC/OC.1', 'OCprim_Deming20%', 'SOC_Deming20%',
       'SOC/OC.2', 'OC1 (µg/m3)', 'OC2 (µg/m3)', 'OC3 (µg/m3)', 'OC4 (µg/m3)',
       'Pyrol (µg/m3)', 'EC1 (µg/m3)', 'EC2 (µg/m3)', 'EC3 (µg/m3)',
       'EC4 (µg/m3)', 'EC5 (µg/m3)', 'EC6 (µg/m3)'],
      dtype='object')


,Id. Filtro,Volumen act (m3),PM2.5 (ug/m3),Periodo Climático,Influencia regional,OC (µg/m3),EC (µg/m3),TC (µg/m3),OC/EC,Percent,...,OC2 (µg/m3),OC3 (µg/m3),OC4 (µg/m3),Pyrol (µg/m3),EC1 (µg/m3),EC2 (µg/m3),EC3 (µg/m3),EC4 (µg/m3),EC5 (µg/m3),EC6 (µg/m3)
FECHA MUESTREO,,,,,,,,,,,,,,,,,,,,,
2019-04-03,2,1712.580544,27.852705,Seca caliente,Incendios agrícolas y forestales,9.676437,1.400565,11.077002,6.908954,NaN,...,2.164393,0.592453,1.012191,2.822556,0.200317,0.687285,2.387941,0.812570,0.105554,0.013736
2019-04-06,3,1709.734574,27.080227,Seca caliente,Incendios agrícolas y forestales,8.995435,0.993862,9.989297,9.050989,NaN,...,1.606222,0.437077,0.896696,3.007266,0.192077,0.744545,2.246232,0.703812,0.094886,0.010994
2019-04-09,4,1698.076137,18.079284,Seca caliente,Incendios agrícolas y forestales,4.566792,0.900457,5.467248,5.071637,SecaCal20%,...,0.755523,0.236342,0.702632,1.049315,0.054205,0.240345,0.984948,0.561526,0.089920,0.008657
2019-04-12,5,1711.655469,23.719727,Seca caliente,Incendios agrícolas y forestales,9.564112,1.125739,10.689851,8.495851,NaN,...,2.013336,0.564331,0.988010,2.865972,0.150240,0.541898,2.140638,0.989453,0.136295,0.017129
2019-04-15,6,1716.985326,39.196608,Seca caliente,Incendios agrícolas y forestales,12.369973,0.756816,13.126790,16.344752,NaN,...,2.489803,0.656622,1.383463,4.355140,0.406755,1.953769,2.431321,0.259018,0.028058,0.015027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-13,119,1650.277055,21.693327,Seca caliente,NaN,7.194447,1.526320,8.720768,4.713590,SecaCal10%,...,1.367475,0.423073,0.833928,1.902229,0.094191,0.308713,1.569045,1.271932,0.151829,0.016329
2020-03-16,120,1624.943598,20.185316,Seca caliente,NaN,4.754171,0.498620,5.252792,9.534660,NaN,...,0.792980,0.292983,0.833862,1.144466,0.067080,0.321357,0.959453,0.248245,0.029256,0.009045
2020-03-19,122,1636.919265,23.336521,Seca caliente,NaN,6.725985,1.499327,8.225314,4.486002,SecaCal10%,...,1.189949,0.366678,0.852717,1.843652,0.094394,0.355998,1.554277,1.187354,0.126864,0.012829


In [30]:
ecuador = pd.read_excel('Datos_Pablo_paraprocesar.xlsx', sheet_name='Ecuador', skiprows=5, nrows=110, usecols='A:BL')
ecuador = ecuador.set_index('Fecha inicial')

meteo_ecuador = pd.read_excel('PROYARCAL-EVENTOS-MUESTREOQUITO_ECUADOR.xlsx', sheet_name='DíasMuestreo', usecols='A:L')
print(meteo_ecuador.keys())
meteo_ecuador['Fecha'] = pd.to_datetime(meteo_ecuador['Fecha'], format="%d/%m/%Y %H:%M").dt.date
meteo_ecuador = meteo_ecuador.set_index('Fecha')
display(meteo_ecuador)


ecuador = pd.concat([ecuador, meteo_ecuador], axis=1)
print(ecuador.keys())
with pd.option_context('display.max_rows', None):
    display(ecuador)
meteo = ['TMP_24h', 'HUM_24h']
medida = ['PM 2.5 (µg/Nm3)', 'OC', 'EC', 'CT', 'OC1', 'OC2', 'OC3', 
         'OC4', 'OC5', 'OC6', 'OC7', 'OC8', 'OP', 'EC1', 'EC2', 
         'EC3', 'EC4', 'EC5', 'EC6', 'OC/EC', 'OC/TC', 'EC/TC', 
         'Ocprim_lin20%', 'SOC_lin20%']

plot_correl(df=ecuador, meteo=meteo, medida=medida, dir_salida='correlaciones/ecuador')


Index(['Fecha', 'RAP_24h', 'DIR_24h', 'HUM_24h', 'LLU_24h', 'PM2.5_24h',
       'PRE_24h', 'RS_24h', 'RS_MAX24h', 'TMP_24h', 'TMPmax24', 'TMPmin24'],
      dtype='object')


,RAP_24h,DIR_24h,HUM_24h,LLU_24h,PM2.5_24h,PRE_24h,RS_24h,RS_MAX24h,TMP_24h,TMPmax24,TMPmin24
Fecha,,,,,,,,,,,
2019-04-03,0.512465,139.195136,84.794583,23.8,20.461667,728.740417,136.453750,713.01,13.132083,16.83,11.64
2019-04-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-04-12,0.262711,122.921082,62.318333,0.3,15.275000,726.439583,238.501667,878.65,16.402917,21.66,12.01
2019-04-15,0.979353,65.225792,69.816522,0.0,15.350000,727.752609,158.336957,738.54,14.707826,18.78,12.16
...,...,...,...,...,...,...,...,...,...,...,...
2020-03-10,0.802395,97.091818,80.023333,28.5,21.973750,728.099167,176.703333,922.90,14.807500,19.87,12.77
2020-03-13,1.119937,93.445514,77.243750,2.3,14.148333,727.515417,227.850833,1066.37,15.070000,19.32,12.04
2020-03-16,1.114885,78.333933,84.677083,9.1,9.008333,727.765000,209.738750,886.35,14.605000,19.07,12.33


Index(['Código ', 'Vol total (Nm3) ', 'PM 2.5 (µg/Nm3)', 'OC', 'EC', 'CT',
       'OC1', 'OC2', 'OC3', 'OC4', 'OC5', 'OC6', 'OC7', 'OC8', 'OP', 'EC1',
       'EC2', 'EC3', 'EC4', 'EC5', 'EC6', 'OC/EC', 'Percent20%',
       'Ocprim_lin20%', 'SOC_lin20%', 'SOC/OC', 'Percent10%', 'Ocprim_lin10%',
       'SOC_lin10%', 'SOC/OC.1', 'Ocprim_Deming20%', 'SOC_Deming20%',
       'SOC/OC.2', 'OC/TC', 'EC/TC', 'OC_prim', 'SOC', 'SOC/OC.3',
       'Unnamed: 39', 'SOC_0,60', 'SOC_0,65', 'SOC_0,70', 'SOC_0,75',
       'SOC_0,80', 'SOC_0,85', 'SOC_0,90', 'SOC_0,95', 'SOC_1,00', 'SOC_1,05',
       'SOC_1,10', 'SOC_1,15', 'SOC_1,20', 'SOC_1,25', 'SOC_1,30', 'SOC_1,35',
       'SOC_1,40', 'SOC_1,45', 'SOC_1,50', 'SOC_1,55', 'SOC_1,60', 'SOC_1,65',
       'SOC_1,70', 'SOC_1,75', 'RAP_24h', 'DIR_24h', 'HUM_24h', 'LLU_24h',
       'PM2.5_24h', 'PRE_24h', 'RS_24h', 'RS_MAX24h', 'TMP_24h', 'TMPmax24',
       'TMPmin24'],
      dtype='object')


,Código,Vol total (Nm3),PM 2.5 (µg/Nm3),OC,EC,CT,OC1,OC2,OC3,OC4,...,DIR_24h,HUM_24h,LLU_24h,PM2.5_24h,PRE_24h,RS_24h,RS_MAX24h,TMP_24h,TMPmax24,TMPmin24
2019-04-03,6001.0,1191.030011,18.735167,4.987650,6.731989,11.719640,1.482086,0.798512,0.507519,1.323985,...,139.195136,84.794583,23.8,20.461667,728.740417,136.453750,713.01,13.132083,16.83,11.64
2019-04-06,6002.0,1187.226494,17.945197,4.060383,4.082750,8.143133,1.311402,0.729391,0.503658,1.511065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-04-09,6003.0,1184.819054,17.543776,4.897393,4.545070,9.442462,1.410919,0.712427,0.396987,1.112064,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-04-12,6004.0,1178.936883,19.402351,5.773567,5.883800,11.657367,1.773098,1.260846,0.836755,2.068506,...,122.921082,62.318333,0.3,15.275000,726.439583,238.501667,878.65,16.402917,21.66,12.01
2019-04-15,6005.0,1183.404461,18.187553,4.179901,4.814241,8.994143,1.438218,0.683233,0.517673,1.586815,...,65.225792,69.816522,0.0,15.350000,727.752609,158.336957,738.54,14.707826,18.78,12.16
2019-04-17,6006.0,1185.544111,19.181962,4.456447,3.987358,8.443806,1.406487,0.655243,0.423705,1.161781,...,120.387140,75.933750,15.1,NaN,726.965417,140.827083,667.34,14.499167,19.21,12.50
2019-04-21,6007.0,1193.427330,11.593155,3.120263,1.863771,4.984034,0.864389,0.428301,0.282472,0.917302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-04-24,6008.0,1188.047573,20.000171,4.402483,5.974342,10.376825,1.167345,0.623399,0.305342,0.784602,...,134.245600,79.810000,2.5,16.090833,728.011250,90.587917,409.07,13.260833,16.06,11.44
2019-04-27,6009.0,1189.944285,17.721036,5.440685,4.289340,9.730025,1.522212,1.008658,0.410220,0.901658,...,241.483925,67.600417,3.8,15.990000,727.657917,166.151250,868.71,14.523333,21.04,11.54
2019-04-29,6010.0,1176.342247,14.293755,3.603553,3.956530,7.560083,1.100542,0.714001,0.301366,0.662028,...,85.751276,61.701667,0.0,12.347917,726.615417,171.423750,790.91,15.672083,19.62,11.82


0

In [4]:
colombia = pd.read_excel('Datos_Pablo_paraprocesar.xlsx', sheet_name='Colombia', skiprows=9, nrows=122, usecols='A:AH')
print(colombia.keys())
colombia = colombia.set_index('Fecha inicial')
# with pd.option_context('display.max_rows', 10):
#     display(colombia)
meteo_colombia = proc_meteo_ish(df=colombia, meteorology_file='ARCAL_ISH/medellin.csv', fechas=colombia.index)
# with pd.option_context('display.max_rows', 10):
#      display(meteo_colombia)

colombia = pd.concat([colombia, meteo_colombia], axis=1)
print(colombia.keys())

meteo = ['tmpd[C]', 'RH[%]']
medida = ['PM2.5 µg/m3', 'OC', 'EC', 'C Total', 'OC1', 'OC2', 'OC3', 
         'OC4', 'OC5', 'EC1', 'EC2', 'EC3', 'EC4', 'EC5', 'EC6',
         'OC/EC', 'OC/CT', 'EC/CT', 'OCprim_lin20%', 'SOC_lin20%']

plot_correl(df=colombia, meteo=meteo, medida=medida, dir_salida='correlaciones/colombia')


Index(['Código ', 'Fecha inicial', 'Volumen ', 'PM2.5 µg/m3', 'OC', 'EC',
       'C Total', 'OC1', 'OC2', 'OC3', 'OC4', 'OC5', 'C_Pyrol', 'EC1', 'EC2',
       'EC3', 'EC4', 'EC5', 'EC6', 'OC1/OC', 'OC/EC', 'Percentil20%',
       'Percentil10%', 'OCprim_lin20%', 'SOC_lin20%', 'SOC/OC',
       'OCprim_lin10%', 'SOC_lin10%', 'SOC/OC.1', 'OCprim_Deming20%',
       'SOC_Deming20%', 'SOC/OC.2', 'OC/CT', 'EC/CT'],
      dtype='object')
Index(['Código ', 'Volumen ', 'PM2.5 µg/m3', 'OC', 'EC', 'C Total', 'OC1',
       'OC2', 'OC3', 'OC4', 'OC5', 'C_Pyrol', 'EC1', 'EC2', 'EC3', 'EC4',
       'EC5', 'EC6', 'OC1/OC', 'OC/EC', 'Percentil20%', 'Percentil10%',
       'OCprim_lin20%', 'SOC_lin20%', 'SOC/OC', 'OCprim_lin10%', 'SOC_lin10%',
       'SOC/OC.1', 'OCprim_Deming20%', 'SOC_Deming20%', 'SOC/OC.2', 'OC/CT',
       'EC/CT', 'wdir', 'wspd[m/s]', 'clht[km]', 'hzvs[km]', 'tmpd[C]',
       'slvp[hPa]', 'press[hPa]', 'sky[octas]', 'skyOpaque[octas]', 'RH[%]',
       'prcp[mm]', 'prcpPeriod[hours]'],


0

In [5]:
costa_rica = pd.read_excel('Datos_Pablo_paraprocesar.xlsx', sheet_name='Costa Rica',
                           skiprows=5, nrows=99, usecols='A:BL')
print(costa_rica.keys())
costa_rica = costa_rica.set_index('Fecha')
with pd.option_context('display.max_rows', 10):
    display(costa_rica)
meteo_costa_rica = proc_meteo_ish(df=costa_rica, meteorology_file='ARCAL_ISH/sanjose.csv', fechas=costa_rica.index)
# with pd.option_context('display.max_rows', 10):
#      display(meteo_costa_rica)

costa_rica = pd.concat([costa_rica, meteo_costa_rica], axis=1)
print(costa_rica.keys())

meteo = ['tmpd[C]', 'RH[%]']
medida = ['PM2,5', 'OC', 'EC', 'CT', 'OC1', 'OC2', 'OC3', 'OC4',
          'OC5', 'OC6', 'OC7', 'OC8', 'OP', 'EC1', 'EC2', 'EC3',
          'EC4', 'EC5', 'EC6', 'OC/EC', 'OC/CT', 'EC/TC', 'OCprim_20%',
          'SOC20%LIN']

plot_correl(df=costa_rica, meteo=meteo, medida=medida, dir_salida='correlaciones/costa_rica')


Index(['Código ', 'Fecha', 'Volumen ', 'PM2,5', 'OC', 'EC', 'CT', 'OC1', 'OC2',
       'OC3', 'OC4', 'OC5', 'OC6', 'OC7', 'OC8', 'OP', 'EC1', 'EC2', 'EC3',
       'EC4', 'EC5', 'EC6', 'OC/EC', 'OC/CT', 'EC/TC', 'Percent10%',
       'Percent20%', 'Percent20%_Seca_sacando_1al6', 'OCprim_10%', 'SOC10%LIN',
       'SOC/OC', 'OCprim_20%', 'SOC20%LIN', 'SOC/OC.1', 'OC_prim', 'SOC',
       'SOC/OC.2', 'Unnamed: 37', 'SOC_0,60', 'SOC_0,65', 'SOC_0,70',
       'SOC_0,75', 'SOC_0,80', 'SOC_0,85', 'SOC_0,90', 'SOC_0,95', 'SOC_1,00',
       'SOC_1,05', 'SOC_1,10', 'SOC_1,15', 'SOC_1,20', 'SOC_1,25', 'SOC_1,30',
       'SOC_1,35', 'SOC_1,40', 'SOC_1,45', 'SOC_1,50', 'SOC_1,55', 'SOC_1,60',
       'SOC_1,65', 'SOC_1,70', 'SOC_1,75', 'SOC_1,80', 'SOC_1,85'],
      dtype='object')


,Código,Volumen,"PM2,5",OC,EC,CT,OC1,OC2,OC3,OC4,...,"SOC_1,40","SOC_1,45","SOC_1,50","SOC_1,55","SOC_1,60","SOC_1,65","SOC_1,70","SOC_1,75","SOC_1,80","SOC_1,85"
Fecha,,,,,,,,,,,,,,,,,,,,,
2019-05-07,35,1436,20.682451,2.725610,2.509340,5.234950,0.974617,0.642655,0.298228,0.497388,...,-0.787466,-0.912933,-1.038400,-1.163868,-1.289335,-1.414802,-1.540269,-1.665736,-1.791203,-1.916670
2019-05-21,40,1427,12.683952,3.994689,4.168189,8.162877,1.165960,0.551621,1.266498,0.043803,...,-1.840776,-2.049185,-2.257595,-2.466004,-2.674414,-2.882823,-3.091233,-3.299642,-3.508051,-3.716461
2019-05-27,42,1421,19.211823,4.007720,3.417933,7.425653,1.218316,0.931432,0.369574,0.670564,...,-0.777386,-0.948282,-1.119179,-1.290076,-1.460972,-1.631869,-1.802766,-1.973662,-2.144559,-2.315456
2019-05-28,43,1476,15.311653,3.465355,2.786969,6.252325,1.022902,0.881371,0.494759,1.202237,...,-0.436402,-0.575750,-0.715099,-0.854447,-0.993796,-1.133144,-1.272493,-1.411841,-1.551189,-1.690538
2019-06-03,45,1329,21.068473,3.721076,2.724277,6.445352,1.063704,1.124457,0.448055,0.731778,...,-0.092911,-0.229125,-0.365339,-0.501553,-0.637767,-0.773981,-0.910194,-1.046408,-1.182622,-1.318836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-21,38,1486,36.810229,6.887599,1.267456,8.155053,1.482957,1.200140,0.425004,0.750234,...,5.113161,5.049788,4.986416,4.923043,4.859670,4.796297,4.732924,4.669552,4.606179,4.542806
2020-04-22,39,1452,30.716253,6.762113,1.307284,8.069399,1.388721,1.907209,0.578973,0.879868,...,4.931916,4.866551,4.801187,4.735823,4.670459,4.605095,4.539730,4.474366,4.409002,4.343638
2020-04-27,40,1481,42.403781,6.071024,1.188716,7.259740,0.858384,1.903843,0.650900,1.029049,...,4.406822,4.347386,4.287950,4.228514,4.169078,4.109643,4.050207,3.990771,3.931335,3.871899


Index(['Código ', 'Volumen ', 'PM2,5', 'OC', 'EC', 'CT', 'OC1', 'OC2', 'OC3',
       'OC4', 'OC5', 'OC6', 'OC7', 'OC8', 'OP', 'EC1', 'EC2', 'EC3', 'EC4',
       'EC5', 'EC6', 'OC/EC', 'OC/CT', 'EC/TC', 'Percent10%', 'Percent20%',
       'Percent20%_Seca_sacando_1al6', 'OCprim_10%', 'SOC10%LIN', 'SOC/OC',
       'OCprim_20%', 'SOC20%LIN', 'SOC/OC.1', 'OC_prim', 'SOC', 'SOC/OC.2',
       'Unnamed: 37', 'SOC_0,60', 'SOC_0,65', 'SOC_0,70', 'SOC_0,75',
       'SOC_0,80', 'SOC_0,85', 'SOC_0,90', 'SOC_0,95', 'SOC_1,00', 'SOC_1,05',
       'SOC_1,10', 'SOC_1,15', 'SOC_1,20', 'SOC_1,25', 'SOC_1,30', 'SOC_1,35',
       'SOC_1,40', 'SOC_1,45', 'SOC_1,50', 'SOC_1,55', 'SOC_1,60', 'SOC_1,65',
       'SOC_1,70', 'SOC_1,75', 'SOC_1,80', 'SOC_1,85', 'wdir', 'wspd[m/s]',
       'clht[km]', 'hzvs[km]', 'tmpd[C]', 'slvp[hPa]', 'press[hPa]',
       'sky[octas]', 'skyOpaque[octas]', 'RH[%]', 'prcp[mm]',
       'prcpPeriod[hours]'],
      dtype='object')


0

In [18]:
brasil = pd.read_excel('Datos_Pablo_paraprocesar.xlsx', sheet_name='Brasil', skiprows=0, nrows=82, usecols='A:V')
print(brasil.keys())
brasil = brasil.set_index('Fecha')
with pd.option_context('display.max_rows', 10):
    display(brasil)
meteo_brasil = proc_meteo_ish(df=brasil, meteorology_file='ARCAL_ISH/medellin.csv', fechas=brasil.index)
# with pd.option_context('display.max_rows', 10):
#      display(meteo_brasil)

brasil = pd.concat([brasil, meteo_brasil], axis=1)
print(brasil.keys())

meteo = ['tmpd[C]', 'RH[%]']
medida = ['PM2.5', 'OC(ug/m3)','EC(ug/m3)', 'TC(ug/m3)', 'OC/EC',
          'OCprim_lin20%', 'SOC_lin20%', 'SOC/OC']

plot_correl(df=brasil, meteo=meteo, medida=medida, dir_salida='correlaciones/brasil')


Index(['Código', 'Fecha', 'Volumen ', 'PM2.5', 'OC(ug/m3)', 'errOC (ug/m3)',
       'EC(ug/m3)', 'errEC (ug/m3)', 'TC(ug/m3)', 'errTC (ug/m3)', 'OC/EC',
       'Percentil 10%', 'Percentil 20%', 'OCprim_lin20%', 'SOC_lin20%',
       'SOC/OC', 'OCprim_lin10%', 'SOC_lin10%', 'SOC/OC.1', 'OCprim_Deming20%',
       'SOC_Deming20%', 'SOC/OC.2'],
      dtype='object')


,Código,Volumen,PM2.5,OC(ug/m3),errOC (ug/m3),EC(ug/m3),errEC (ug/m3),TC(ug/m3),errTC (ug/m3),OC/EC,...,Percentil 20%,OCprim_lin20%,SOC_lin20%,SOC/OC,OCprim_lin10%,SOC_lin10%,SOC/OC.1,OCprim_Deming20%,SOC_Deming20%,SOC/OC.2
Fecha,,,,,,,,,,,,,,,,,,,,,
2019-04-03,1000,1445.54,35.26,12.263843,NaN,5.179119,NaN,NaN,NaN,2.367940,...,0,8.038579,4.225264,0.344530,7.879693,4.384150,0.357486,8.041176,4.222667,0.344318
2019-04-06,1001,1448.11,37.22,5.734971,NaN,0.923710,NaN,NaN,NaN,6.208628,...,0,1.728933,4.006038,0.698528,1.922933,3.812037,0.664700,NaN,NaN,NaN
2019-04-09,1002,1450.99,36.28,12.046084,NaN,0.515257,NaN,NaN,NaN,23.378801,...,0,1.123305,10.922779,0.906749,1.351177,10.694907,0.887833,NaN,NaN,NaN
2019-04-12,1003,1449.45,28.08,6.294896,NaN,1.795752,NaN,NaN,NaN,3.505438,...,0,3.021940,3.272956,0.519938,3.143625,3.151271,0.500607,NaN,NaN,NaN
2019-04-15,1004,1458.05,18.18,3.304629,NaN,1.165341,NaN,NaN,NaN,2.835761,...,0,2.087208,1.217420,0.368399,2.261171,1.043457,0.315756,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-14,1085,1487.23,24.07,5.259771,NaN,0.436760,NaN,NaN,NaN,12.042694,...,0,1.562272,3.697499,0.702977,1.553367,3.706404,0.704670,NaN,NaN,NaN
2019-12-20,1087,1474.04,28.97,6.970405,NaN,1.106483,NaN,NaN,NaN,6.299606,...,0,4.013020,2.957384,0.424277,3.695739,3.274665,0.469796,NaN,NaN,NaN
2019-12-26,1089,1473.86,29.79,3.680835,NaN,0.506653,NaN,NaN,NaN,7.265001,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Index(['Código', 'Volumen ', 'PM2.5', 'OC(ug/m3)', 'errOC (ug/m3)',
       'EC(ug/m3)', 'errEC (ug/m3)', 'TC(ug/m3)', 'errTC (ug/m3)', 'OC/EC',
       'Percentil 10%', 'Percentil 20%', 'OCprim_lin20%', 'SOC_lin20%',
       'SOC/OC', 'OCprim_lin10%', 'SOC_lin10%', 'SOC/OC.1', 'OCprim_Deming20%',
       'SOC_Deming20%', 'SOC/OC.2', 'wdir', 'wspd[m/s]', 'clht[km]',
       'hzvs[km]', 'tmpd[C]', 'slvp[hPa]', 'press[hPa]', 'sky[octas]',
       'skyOpaque[octas]', 'RH[%]', 'prcp[mm]', 'prcpPeriod[hours]'],
      dtype='object')


0